# Q1

In [1]:
from bs4 import BeautifulSoup
import requests
page = requests.get("https://en.wikipedia.org/wiki/Main_Page")
print (page) #if response is 200 : page is accessing 
print (page.status_code)
soup = BeautifulSoup(page.content, 'html.parser')
header_tags = soup.find_all(['h1', 'h2','h3','h4','h5','h6'])
print ('All header tags in wikipedia main page are: ', *header_tags)

<Response [200]>
200
All header tags in wikipedia main page are:  <h1 class="firstHeading" id="firstHeading">Main Page</h1> <h2 class="mp-h2" id="mp-tfa-h2"><span id="From_today.27s_featured_article"></span><span class="mw-headline" id="From_today's_featured_article">From today's featured article</span></h2> <h2 class="mp-h2" id="mp-dyk-h2"><span class="mw-headline" id="Did_you_know_...">Did you know ...</span></h2> <h2 class="mp-h2" id="mp-itn-h2"><span class="mw-headline" id="In_the_news">In the news</span></h2> <h2 class="mp-h2" id="mp-otd-h2"><span class="mw-headline" id="On_this_day">On this day</span></h2> <h2 class="mp-h2" id="mp-tfp-h2"><span id="Today.27s_featured_picture"></span><span class="mw-headline" id="Today's_featured_picture">Today's featured picture</span></h2> <h2 class="mp-h2" id="mp-other"><span class="mw-headline" id="Other_areas_of_Wikipedia">Other areas of Wikipedia</span></h2> <h2 class="mp-h2" id="mp-sister"><span id="Wikipedia.27s_sister_projects"></span><sp

# Q2 and Q3 : Same question repeated

In [18]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
url = 'http://www.imdb.com/chart/top'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
#print (soup)
movies = soup.select('td.titleColumn')
ratings = [b.attrs.get('data-value') for b in soup.select('td.posterColumn span[name=ir]')]
votes = [b.attrs.get('data-value') for b in soup.select('td.ratingColumn strong')]

imdb = []
for index in range(0, 100):
    # Seperate movie into: 'place', 'title', 'year'
    movie_string = movies[index].get_text()
    movie = (' '.join(movie_string.split()).replace('.', ''))
    movie_title = movie[len(str(index))+1:-7]
    year = re.search('\((.*?)\)', movie_string).group(1)
    
    data = {"movie_title": movie_title,
            "year": year,
            "rating": ratings[index]}
    imdb.append(data)
imdb[0]
df1= pd.DataFrame(imdb)
df1.head()

,movie_title,year,rating
0,The Shawshank Redemption,1994,9.220966523832836
1,The Godfather,1972,9.147121195568806
2,The Godfather: Part II,1974,8.981057198511023
3,The Dark Knight,2008,8.975579288613655
4,12 Angry Men,1957,8.94176781884409


# Q4

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
url = 'https://meesho.com/bags-ladies/pl/p7vbp'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html')
#print (soup)
print (soup.findAll())


[<html lang="en"><head><script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-113318580-2"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-113318580-6"></script><script async="">
              window.dataLayer = window.dataLayer || [];
              function gtag(){dataLayer.push(arguments);}
              gtag('js', new Date());

              gtag('config', 'UA-113318580-2', {
                page_path: window.location.pathname,
              });
          </script><script async="" src="https://www.googletagmanager.com/gtag/js?id=G-ZB97HR591S"></script><script async="">
              window.dataLayer = window.dataLayer || [];
              function gtag(){dataLayer.push(arguments);}
              gtag('js', new Date());

              gtag('config', 'G-ZB97HR591S', {
                page_path: window.location.pathname,
              });
          </script><script async="">
              window.dataLayer = window.dataLayer || [];
           

# Q5

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
url = 'https://www.icc-cricket.com/rankings/mens/player-rankings/test'
page =requests.get(url)
soup = BeautifulSoup(page.text, 'html')
table = soup.find_all('tbody')

for i, j in enumerate(table):
    print ('rank is: ', i, j)
    
table_body=soup.find_all('tbody')
batsmen_df = pd.DataFrame(columns=["Name","Desc","Runs", "Balls", "4s", "6s", "SR", "Team"])
for i, table in enumerate(table_body[0:4:2]):
    rows = table.find_all('tr')
    for row in rows[::2]:
        cols=row.find_all('td')
        cols=[x.text.strip() for x in cols]
        if cols[0] == 'Extras':
            continue
        if len(cols) > 7:
            batsmen_df = batsmen_df.append(pd.Series(
            [re.sub(r"\W+", ' ', cols[0].split("(c)")[0]).strip(), cols[1], 
            cols[2], cols[3], cols[5], cols[6], cols[7], i+1], 
            index=batsmen_df.columns ), ignore_index=True)
        else:
            batsmen_df = batsmen_df.append(pd.Series(
            [re.sub(r"\W+", ' ', cols[0].split("(c)")[0]).strip(), cols[1], 
            0, 0, 0, 0, 0, i+1], index = batsmen_df.columns), ignore_index=True)
                
    for i in range(2):
        dnb_row = soup.find_all("tfoot")[i].find_all("div")
        for c in dnb_row:
            dnb_cols = c.find_all('span')
            dnb = [x.text.strip().split("(c)")[0] for x in dnb_cols]
            dnb = filter(lambda item: item, [re.sub(r"\W+", ' ', x).strip() for x in dnb])
            for dnb_batsman in dnb:
                batsmen_df = batsmen_df.append(pd.Series([dnb_batsman, "DNB", 0, 0, 0, 0, 0, i+1], index = batsmen_df.columns), ignore_index =True)

batsmen_df    

rank is:  0 <tbody>
<tr class="table-body">
<td class="table-body__cell table-body__cell--position u-text-right">
<div class="rankings-table__pos-container">
<span class="rankings-table__pos-number">
                                    2
                                </span>
<div class="ranking-pos down">
<svg class="icon">
<use xlink:href="/resources/prod/v8.21.1/i/svg-output/icons.svg#icn-chevron-down" xmlns:xlink="http://www.w3.org/1999/xlink"></use>
</svg>
<span class="ranking-pos__change-amount">(1)</span>
<span class="ranking-pos__tooltip">This player has moved down in the rankings since the previous rankings update</span>
</div>
</div>
</td>
<td class="table-body__cell name">
<a href="/rankings/mens/player-rankings/887">Joe Root</a>
</td>
<td class="table-body__cell nationality-logo">
<span class="flag-15 table-body_logo ENG"></span>
<span class="table-body__logo-text">ENG</span>
</td>
<td class="table-body__cell u-text-right rating">897</td>
</tr>
<tr class="table-body">
<td 

C:\Users\VARMA\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\VARMA\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


IndexError: list index out of range

# Q6

In [11]:
def extract_bowling_data(series_id, match_id):

    URL = 'https://www.espncricinfo.com/series/'+ str(series_id) + '/scorecard/' + str(match_id)
    page = requests.get(URL)
    bs = BeautifulSoup(page.content, 'lxml')

    table_body=bs.find_all('tbody')
    bowler_df = pd.DataFrame(columns=['Name', 'Overs', 'Maidens', 'Runs', 'Wickets',
                                      'Econ', 'Dots', '4s', '6s', 'Wd', 'Nb','Team'])
    for i, table in enumerate(table_body[1:4:2]):
        rows = table.find_all('tr')
        for row in rows:
            cols=row.find_all('td')
            cols=[x.text.strip() for x in cols]
            bowler_df = bowler_df.append(pd.Series([cols[0], cols[1], cols[2], cols[3], cols[4], cols[5], 
                                                    cols[6], cols[7], cols[8], cols[9], cols[10], (i==0)+1], 
                                                   index=bowler_df.columns ), ignore_index=True)
    return bowler_df
extract_bowling_data(series_id = 8048, match_id = 1136561)

IndexError: list index out of range

# Q7

In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
url = 'https://coreyms.com/'
page =requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')
soup.find_all('div')[0]

<div class="site-container"><header class="site-header" itemscope="" itemtype="https://schema.org/WPHeader"><div class="wrap"><div class="title-area"> <div class="site-avatar">
<a href="https://coreyms.com/"><svg class="site-avatar-svg" enable-background="new 0 0 441.5 441.5" height="150px" id="Layer_1" version="1.1" viewbox="0 0 441.5 441.5" width="150px" x="0px" xml:space="preserve" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" y="0px">
<g class="site-avatar-background">
<path d="M79.178 390.133C30.781 349.639 0 288.789 0 220.75C0 98.833 98.833 0 220.75 0S441.5 98.833 441.5 220.75 c0 63.558-26.86 120.842-69.848 161.12" fill="#56616B"></path>
</g>
<g class="site-avatar-foreground">
<path d="M254.602 182.291c0 1.992-0.097 4 0 6c0.057 0.88-0.093 1.952 0.194 2.78c0.22 0.631 0.69 1.12 1.704 1.273 c2.009 0.3 3.436-1.062 4.384-2.719c0.712-1.244 0.863-3.376 1.843-3.807c1.612-0.712 2.646-1.537 3.276-2.44 c1.903-2.732 0.09-6.185-0.723-9.42c-0.29-1.157-1.995-2.03

# Q9

In [27]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
url = 'https://www.dineout.co.in/delhi-restaurants/buffet-special'
page =requests.get(url)
soup = BeautifulSoup(page.text, 'html')
list1=[]
for i in soup.find_all('li'):
    print (i)

<li class="loc-detection" data-w-onclick="locateMeHandler|w0-w0-searchLocationList"><a class="btn btn-default" href="javascript:void(0)" id="locDec"><span class="ellipsis">Use my current location</span><i class="do do-location-detection"></i></a></li>
<li class="login" data-w-events="w0,openLogin,openLoginPopup" data-widget="/dineout.co.in$0.0.2/girf/components/header-desktop-user/index" id="w0-desktopuser"><a class="dflt_color d-loginbtn" href="javascript:void(0);" onclick="open_login_redirect('');">Log in</a></li>
<li><a data-action="Home" data-w-onclick="headerListClick|w0" href="/">Home</a></li>
<li><a data-action="bookATable" data-w-onclick="headerListClick|w0" href="/delhi-restaurants">Book a Table</a></li>
<li><a data-action="DineoutPay" data-w-onclick="headerListClick|w0" href="/delhi-restaurants/dineout-pay">Dineout Pay</a></li>
<li><a data-action="GourmetPassport" data-w-onclick="headerListClick|w0" href="/delhi/dineout-passport">Dineout Passport</a></li>
<li><a data-action="

C:\Users\VARMA\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\VARMA\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
